# Subqueries

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-01-25

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om subqueries i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16)


## Indholdsfortegnelse

- [Introduktion til subqueries](#Introduktion-til-subqueries)
- [Uafhængige subqueries](#Uafhængige-subqueries)
  - [Skalar subquery](#Skalar-subquery)
  - [Flerværdi subquery](#Flerværdi-subquery)
- [Korrelerede subqueries](#Korrelerede-subqueries)
  - [EXISTS-prædikatet](#EXISTS-prædikatet)
- [Hovedpointer](#Hovedpointer)


## Introduktion til subqueries

SQL understøtter at man kan indlejre forespørgsler i hinanden.

Man siger at den forespørgsel hvis resultat returneres til forespørgeren er den *ydre* forespørgsel, mens den *indre* forespørgsel er en der bruges af den ydre forespørgsel. Sidstnævnte kaldes en *subquery*.

Man kan tænke på at subqueries står i stedet for et udtryk baseret på konstanter eller variable som evalueres på kørselstidspunktet. Subqueries er dynamiske i sin natur, dvs. resultatet af en subquery vil ændre sig i takt med at indholdet i de underliggende tabeller ændrer sig.

Subqueries kan have nogle forskellige egenskaber:

- De kan være både uafhængige eller korreleret med den ydre forespørgsel
- De kan returnere en enkelt værdi, en liste af værdier eller en hel tabel
 
 Mens subqueries der returnerer hele tabeller vil gennemgås i emnet om [tabeludtryk](Table-expressions.ipynb), vil de øvrige typer af subqueries gennemgås i nærværende notebook.

Use cases:

- Dynamisk beregningstrin i forespørgsel
- Beregning af positiv- og negativlister til filtrering


In [ ]:
/* Eksempel */

## Uafhængige subqueries

Uafhængige subqueries er *subqueries* som er *uafhængige* af tabellerne fra den ydre forespørgsel som indeholder den. De evalueres logisk før den ydre forespørgsel, og den ydre forspørgsel bruger resultatet fra den indre forespørgsel.

Uafhængige subqueries kan returnere en [enkelt værdi](#Skalar-subquery) eller en [liste af værdier](#Flerværdi-subquery).


### Skalar subquery

En skalar subquery er en subquery som returnerer en enkelt værdi.

En skalar subquery må optræde over alt i den ydre forespørgsel hvor udtryk med enkeltværdier er tilladt, fx `SELECT` og `WHERE`.


In [ ]:
/* Eksempel */

/* Bemærk at uafhængige subqueries kan markeres og eksekveres */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


### Flerværdi subquery

En flerværdi subquery er en subquery som returnerer flere værdier som en enkelt kolonne, eller en liste af værdier.

En flerværdi subquery bruges i nogle prædikater, fx `IN` (samt `SOME`, `ANY` og `ALL`, men disse bruges sjældent).

`IN`-prædikatet har formen:

```sql
SELECT
    *
FROM TableA
WHERE <skalar-udtryk> IN (<flerværdi subquery>);
```

Prædikatet evalueres til *SANDT* såfremt skalar-udtrykket er lig en af værdierne returneret af subquerien.


In [ ]:
/* Eksempel */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


## Korrelerede subqueries

Korrelerede subqueries er subqueries som refererer tabeller og kolonner fra den ydre forespørgsel. Korrelerede subqueries er således afhængige af den ydre forespørgsel og kan ikke eksekveres for sig. De evalueres logisk separat for hver række i den ydre forespørgsel.


In [ ]:
/* Eksempel */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


### `EXISTS`-prædikatet

T-SQL understøtter et prædikat kaldt `EXISTS` som tager en (korreleret) subquery som input og returnerer *SANDT* hvis subquerien returnerer en eller flere rækker (matches) og ellers *FALSKT*.

`EXISTS`-prædikatet har formen:

```sql
SELECT
    *
FROM TableA
WHERE EXISTS (
    <korreleret subquery>
);
```


In [ ]:
/* Eksempel */

/* EXISTS performer godt! Databasesystemet ved godt at det udelukkende skal finde ud af om
der eksisterer mindst en matchende række eller ingen */

/* Mens brugen af asterisk, *, oftest er en dårlig praksis, så gælder dette ikke
for EXISTS - her bekymrer databasesystemet sig udelukkende om matchende rækker */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


## Hovedpointer

- Subqueries er en forespørgsel indlejret i en anden forespørgsel
- Subqueries kan have forskellige egenskaber. De kan være:
  - Uafhængige eller korrelede
  - Returnere en enkelt værdi eller en liste af værdier
- Subqueries bruges ofte fordi de er dynamiske og sparer nogle beregningstrin


## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
